In [1]:
from keras.models import load_model
import joblib
from helpers import *
import numpy as np 
import pandas as pd
from flask import Flask , render_template
from flask import request

In [2]:
def base_model():
    #Create the model
    model = Sequential()
    #Add 1 layer with 8 nodes,input of 4 dim with relu function
    model.add(Dense(8,input_dim=10,activation='relu'))
    #Add 1 layer with output 3 and softmax function
    model.add(Dense(4,activation='softmax'))
    #Compile the model using sigmoid loss function and adam optim
    model.compile(loss='categorical_crossentropy',optimizer='adam',
                 metrics=['accuracy'])
    return model

In [3]:
model = joblib.load('sklearn_pipeline.pkl')
model.named_steps['keras'].model = load_model('model.h5')

In [4]:
target = pd.read_csv('target.csv')

In [5]:
def predict_mood(id_song):

    preds = get_songs_features(id_song)
    #Pre-process the features to input the Model
    preds_features = np.array(preds[0][6:-2]).reshape(-1,1).T

    #Predict the features of the song
    results = model.predict(preds_features)
    print(results)
    mood = np.array(target['mood'][target['encode']==int(results)])
    name_song = preds[0][0]
    artist = preds[0][2]
    print("{0} by {1} is a {2} song".format(name_song,artist,mood[0].upper()))
    final_data = preds[0] + list(mood) 
    print(final_data)
    return final_data

In [6]:
def input_id():
    data_dict = request.form
    song_id = data_dict['Song_ID']
#     artist_id = data_dict['Artist_ID']
#     playlist_id = data_dict['Playlist_ID']
    return predict_mood(song_id)

In [7]:
app =Flask(__name__)

@app.route("/")
def info():
    return render_template('info.html')


In [8]:
@app.route("/predict", methods=['POST'])
def predict():
    final_data = input_id()
    return render_template("index.html", info=final_data)

In [ ]:
if __name__ == '__main__':
#     load_model()
#     app.run(host='0.0.0.0', port=8000)
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Nov/2020 00:23:22] "GET / HTTP/1.1" 200 -


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


127.0.0.1 - - [17/Nov/2020 00:23:25] "POST /predict HTTP/1.1" 200 -


[1]
Blinding Lights by The Weeknd is a ENERGETIC song
['Blinding Lights', 'After Hours', 'The Weeknd', '0VjIjW4GlUZAMYd2vXMi3b', '2020-03-20', 96, 200040, 0.514, 0.00146, 0.73, 9.54e-05, 0.0897, 0.334, -5.934, 0.0598, 171.005, 1, 4, 'Energetic']
